**The code snippet you provided only imports the necessary modules and does not include any specific functionality or operations. It serves as a prerequisite for using the modules in further code that may follow.**

In [ ]:
from pathlib import Path
import time
import os
from google.cloud import bigquery

**This code sets the Google Cloud Platform (GCP) service account credentials, project ID, dataset ID, and table ID using environment variables. It then establishes a connection to Google BigQuery, executes a SQL query to select all rows from the specified table, and prints each row in the query result.**

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.environ['GCP_CRED']
project_id = os.environ['project_id']
dataset_id = os.environ['dataset_id']
table_id = os.environ['table_id']

client = bigquery.Client()
data_file_folder = os.environ['dataset_path']

sql_query = f"SELECT * FROM {project_id}.{dataset_id}.{table_id}"

query_job = client.query(sql_query)

for row in query_job.result():
	print(row)

**This code defines a function called `table_reference` that takes in a project ID, dataset ID, and table ID as parameters. It creates a dataset reference and a table reference using the provided IDs, then prints the table reference and returns it as the output of the function.**

In [ ]:
def table_reference(project_id, dataset_id, table_id):
    dataset_ref = bigquery.DatasetReference(project_id, dataset_id)
    table_ref = bigquery.TableReference(dataset_ref, table_id)
    print(table_ref)
    return table_ref

**This code defines a function called `delete_dataset_tables` that takes a project ID and dataset ID as parameters. It uses the BigQuery client to retrieve a list of tables in the specified dataset, then iterates over each table and deletes it. Finally, it prints a message indicating that the tables have been deleted.**

In [ ]:
def delete_dataset_tables(project_id, dataset_id):
    tables = client.list_tables(f'{project_id}.{dataset_id}')
    for table in tables:
        client.delete_table(table)
    print('Tables deleted.')

**This code defines a function called `upload_csv` that uploads a CSV file into a specified table in Google BigQuery. It configures the schema for the table and the load job settings, then uses the BigQuery client to load the CSV file data into the table. It monitors the status of the upload job until it is done, then prints the result of the upload.**

In [21]:
def upload_csv(client, table_ref, csv_file):
    # client.delete_table(table_ref, not_found_ok=True)

    load_job_configuration = bigquery.LoadJobConfig()
    load_job_configuration.schema = [
        bigquery.SchemaField('Lead_ID', 'STRING', mode='NULLABLE'),
        bigquery.SchemaField('Entry_Date', 'DATE', mode='NULLABLE'),
        bigquery.SchemaField('Modify_Date', 'DATE', mode='NULLABLE'),
        bigquery.SchemaField('Hours_Called', 'FLOAT', mode='NULLABLE'),
        bigquery.SchemaField('Project_Cost_Hourly_Rate', 'FLOAT', mode='NULLABLE'),
        bigquery.SchemaField('Campaign', 'STRING', mode='REQUIRED'),
        bigquery.SchemaField('Call_Status', 'STRING', mode='REQUIRED'),
        bigquery.SchemaField('New_Status', 'STRING', mode='REQUIRED'),
        bigquery.SchemaField('Contact_Status', 'STRING', mode='REQUIRED'),
        bigquery.SchemaField('Call_Stagging', 'DATE', mode='NULLABLE'),
        bigquery.SchemaField('Total_Hours_Called', 'FLOAT', mode='NULLABLE'),
        bigquery.SchemaField('Project_Cost_Rate', 'FLOAT', mode='NULLABLE'),
        bigquery.SchemaField('Working_Days', 'FLOAT', mode='NULLABLE'),
        bigquery.SchemaField('Registration_Goal', 'FLOAT', mode='NULLABLE'),
        bigquery.SchemaField('Campaign_Status', 'STRING', mode='REQUIRED')
    ]

    # load_job_configuration.autodetect = True
    load_job_configuration.source_format = bigquery.SourceFormat.CSV
    load_job_configuration.skip_leading_rows = 1
    load_job_configuration.allow_quoted_newlines = True

    with open(csv_file, 'rb') as source_file:
        upload_job = client.load_table_from_file(
            source_file,
            destination=table_ref,          
            location='US',
            job_config=load_job_configuration
        )

    while upload_job.state != 'DONE':
        time.sleep(2)
        upload_job.reload()
        print(upload_job.state)
    print(upload_job.result())

**This code invokes the function `delete_dataset_tables` with the provided `project_id` and `dataset_id` as arguments. The function deletes all the tables within the specified dataset in Google BigQuery associated with the given project ID.**

In [ ]:
delete_dataset_tables(project_id, dataset_id)

**This code iterates over the files within the `data_file_folder` directory. It prints the name of each file and if a file starts with '2023', it processes it by creating a table reference based on the file name, then uploads the corresponding CSV file into the specified table using the `upload_csv` function.**

In [ ]:
for file in os.listdir(data_file_folder):
    print(file)
    if file.startswith('2023'):
        print('Processing file: {0}'.format(file))
        table_name = '_'.join(file.split('.')[:2])
        csv_file = data_file_folder.joinpath(file)
        print(csv_file)
        table_ref = table_reference(project_id, dataset_id, table_name)
        upload_csv(client, table_ref, csv_file)
        print()